In [1]:
import re
import difflib
import numpy as np
import pandas as pd

In [2]:
full_data = pd.read_excel('./Vecindarios_Alimentarios_Encuesta_Integrada_2022_11_23.xlsx', sheet_name=None)

In [3]:
for name in full_data:
    print(name)

Vecindarios Alimentarios - E...
composicion_hogar
_5_18_medidas_control_plagas
_5_19_productos_control_plagas
_5_20_control_malezas
_7_1_Oferta_Alimentos_Produccio
_7_2_produc_dejo_producir_vend
Productos_Vende_Canal
canales_venta_productores
aplicacion_biopreparados
personas_trabajan_puesto_venta
canales_compra_comerciantes
Productos_Compra_Canal
APOYO_RECIBIDO
APOYO_BRINDADO
HUERTO_RELACIONES


In [4]:
columns_to_drop = ['_index', '_parent_table_name', '_submission__id', '_submission__uuid', '_submission__submission_time', '_submission__validation_status', '_submission__notes', '_submission__status', '_submission__submitted_by', '_submission__tags']

In [5]:
vecindarios = full_data['Vecindarios Alimentarios - E...']
composicion_hogar = full_data['composicion_hogar'].drop(columns=columns_to_drop)
medidas_control_plagas = full_data['_5_18_medidas_control_plagas'].drop(columns=columns_to_drop)
productos_control_plagas = full_data['_5_19_productos_control_plagas'].drop(columns=columns_to_drop)
control_malezas = full_data['_5_20_control_malezas'].drop(columns=columns_to_drop)
oferta_alimentos = full_data['_7_1_Oferta_Alimentos_Produccio'].drop(columns=columns_to_drop)
producto_dejo_producir = full_data['_7_2_produc_dejo_producir_vend'].drop(columns=columns_to_drop)
producto_vende_canal = full_data['Productos_Vende_Canal'].drop(columns=columns_to_drop)
canales_venta_productores = full_data['canales_venta_productores'].drop(columns=columns_to_drop)
aplicacion_biopreparados = full_data['aplicacion_biopreparados'].drop(columns=columns_to_drop)
personas_trabajan_puesto_venta = full_data['personas_trabajan_puesto_venta'].drop(columns=columns_to_drop)
canales_compra_comerciantes = full_data['canales_compra_comerciantes'].drop(columns=columns_to_drop)
productos_compra_canal = full_data['Productos_Compra_Canal'].drop(columns=columns_to_drop)
APOYO_RECIBIDO = full_data['APOYO_RECIBIDO'].drop(columns=columns_to_drop)
APOYO_BRINDADO = full_data['APOYO_BRINDADO'].drop(columns=columns_to_drop)
HUERTO_RELACIONES = full_data['HUERTO_RELACIONES'].drop(columns=columns_to_drop)

In [6]:
pd.merge(left=vecindarios, right=composicion_hogar, left_on='_index', right_on='_parent_index')

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,_status,_submitted_by,_tags,_parent_index,miembro_parentesco,otro_pariente,otro_NO_pariente,sexo_y,edad_miembro,miembros_aporta_economicamente
0,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,submitted_via_web,gaparedes,NaN,1,el_entrevistado,NaN,NaN,femenino,54.0,si
1,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,submitted_via_web,gaparedes,NaN,1,esposos,NaN,NaN,masculino,54.0,si
2,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,submitted_via_web,gaparedes,NaN,1,hijos_hijastros,NaN,NaN,masculino,22.0,no
3,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,submitted_via_web,gaparedes,NaN,1,hijos_hijastros,NaN,NaN,masculino,25.0,no
4,2,2022-07-29 21:33:38.674,2022-07-30 20:44:17.615999,NaN,2022-07-30,Fabian Sevilla,quito,solanda,NaN,productor,...,submitted_via_web,fabian1127,NaN,2,el_entrevistado,NaN,NaN,femenino,54.0,si
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3202,894,2022-10-30 17:00:24.259,2022-10-30 18:02:45.276000,NaN,2022-06-17,Jaime Perez Salinas,lima,fuera,no puso,productor,...,submitted_via_web,jorgereyna,NaN,894,padres,NaN,NaN,femenino,60.0,si
3203,895,2022-10-30 18:02:45.437,2022-10-30 18:44:29.245000,NaN,2022-10-02,Anny Diana Gomez de la Cruz,lima,fuera,Cieneguilla,productor,...,submitted_via_web,jorgereyna,NaN,895,el_entrevistado,NaN,NaN,masculino,66.0,si
3204,895,2022-10-30 18:02:45.437,2022-10-30 18:44:29.245000,NaN,2022-10-02,Anny Diana Gomez de la Cruz,lima,fuera,Cieneguilla,productor,...,submitted_via_web,jorgereyna,NaN,895,esposos,NaN,NaN,femenino,65.0,no
3205,895,2022-10-30 18:02:45.437,2022-10-30 18:44:29.245000,NaN,2022-10-02,Anny Diana Gomez de la Cruz,lima,fuera,Cieneguilla,productor,...,submitted_via_web,jorgereyna,NaN,895,hijos_hijastros,NaN,NaN,masculino,32.0,no


In [7]:
vecindarios_productor = vecindarios.loc[vecindarios['_1_5_perfil_entrevistado']=='productor'].reset_index(drop=True)

In [8]:
vecindarios_comerciante = vecindarios.loc[vecindarios['_1_5_perfil_entrevistado']=='comerciante'].reset_index(drop=True)

In [9]:
vecindarios_consumidor = vecindarios.loc[vecindarios['_1_5_perfil_entrevistado']=='consumidor'].reset_index(drop=True)

In [50]:
df_productor = pd.merge(left=vecindarios_productor, right=composicion_hogar, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=medidas_control_plagas, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=control_malezas, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=oferta_alimentos, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=producto_dejo_producir, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=producto_vende_canal, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=canales_venta_productores, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=aplicacion_biopreparados, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=personas_trabajan_puesto_venta, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=canales_compra_comerciantes, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=productos_compra_canal, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=APOYO_RECIBIDO, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=APOYO_BRINDADO, left_on='_index', right_on='_parent_index', how='left')
df_productor = pd.merge(left=df_productor, right=HUERTO_RELACIONES, left_on='_index', right_on='_parent_index', how='left')

C:\Users\norte\AppData\Local\Temp\ipykernel_6772\2927134885.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'_parent_index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_productor = pd.merge(left=df_productor, right=oferta_alimentos, left_on='_index', right_on='_parent_index', how='left')
C:\Users\norte\AppData\Local\Temp\ipykernel_6772\2927134885.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'_parent_index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_productor = pd.merge(left=df_productor, right=producto_vende_canal, left_on='_index', right_on='_parent_index', how='left')
C:\Users\norte\AppData\Local\Temp\ipykernel_6772\2927134885.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'_parent_index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_productor = pd.merge(left=df_productor, right=aplicacion_bio

In [51]:
df_productor

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,huerto_tipo_relacion/interc_produccion,huerto_tipo_relacion/interc_insumos,huerto_tipo_relacion/acopio_comercio,huerto_tipo_relacion/ensenanza,huerto_tipo_relacion/otra_relacion,huerto_otra_relacion,_2019_huerto_relacion,_2022_huerto_relacion,huertos_observaciones,_parent_index_y
0,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19620,894,2022-10-30 17:00:24.259,2022-10-30 18:02:45.276000,NaN,2022-06-17,Jaime Perez Salinas,lima,fuera,no puso,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19621,895,2022-10-30 18:02:45.437,2022-10-30 18:44:29.245000,NaN,2022-10-02,Anny Diana Gomez de la Cruz,lima,fuera,Cieneguilla,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19622,895,2022-10-30 18:02:45.437,2022-10-30 18:44:29.245000,NaN,2022-10-02,Anny Diana Gomez de la Cruz,lima,fuera,Cieneguilla,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19623,895,2022-10-30 18:02:45.437,2022-10-30 18:44:29.245000,NaN,2022-10-02,Anny Diana Gomez de la Cruz,lima,fuera,Cieneguilla,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df_comerciante = pd.merge(left=vecindarios_comerciante, right=composicion_hogar, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=medidas_control_plagas, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=control_malezas, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=oferta_alimentos, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=producto_dejo_producir, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=producto_vende_canal, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=canales_venta_productores, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=aplicacion_biopreparados, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=personas_trabajan_puesto_venta, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=canales_compra_comerciantes, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=productos_compra_canal, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=APOYO_RECIBIDO, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=APOYO_BRINDADO, left_on='_index', right_on='_parent_index', how='left')
df_comerciante = pd.merge(left=df_comerciante, right=HUERTO_RELACIONES, left_on='_index', right_on='_parent_index', how='left')

C:\Users\norte\AppData\Local\Temp\ipykernel_6772\1830908221.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'_parent_index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_comerciante = pd.merge(left=df_comerciante, right=oferta_alimentos, left_on='_index', right_on='_parent_index', how='left')
C:\Users\norte\AppData\Local\Temp\ipykernel_6772\1830908221.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'_parent_index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_comerciante = pd.merge(left=df_comerciante, right=producto_vende_canal, left_on='_index', right_on='_parent_index', how='left')
C:\Users\norte\AppData\Local\Temp\ipykernel_6772\1830908221.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'_parent_index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_comerciante = pd.merge(left=df_comerciante, right=ap

In [53]:
df_comerciante

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,huerto_tipo_relacion/interc_produccion,huerto_tipo_relacion/interc_insumos,huerto_tipo_relacion/acopio_comercio,huerto_tipo_relacion/ensenanza,huerto_tipo_relacion/otra_relacion,huerto_otra_relacion,_2019_huerto_relacion,_2022_huerto_relacion,huertos_observaciones,_parent_index_y
0,24,2022-08-10 15:06:32.767,2022-08-10 18:57:42.942,NaN,2022-06-25,JAFET WILLIAMS PARIONA AVALOS,lima,carabayllo,NaN,comerciante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24,2022-08-10 15:06:32.767,2022-08-10 18:57:42.942,NaN,2022-06-25,JAFET WILLIAMS PARIONA AVALOS,lima,carabayllo,NaN,comerciante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24,2022-08-10 15:06:32.767,2022-08-10 18:57:42.942,NaN,2022-06-25,JAFET WILLIAMS PARIONA AVALOS,lima,carabayllo,NaN,comerciante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24,2022-08-10 15:06:32.767,2022-08-10 18:57:42.942,NaN,2022-06-25,JAFET WILLIAMS PARIONA AVALOS,lima,carabayllo,NaN,comerciante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24,2022-08-10 15:06:32.767,2022-08-10 18:57:42.942,NaN,2022-06-25,JAFET WILLIAMS PARIONA AVALOS,lima,carabayllo,NaN,comerciante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689,822,2022-10-06 19:48:09.238,2022-10-06 20:02:32.385,NaN,2022-09-15,Sandra Rivera,quito,solanda,NaN,comerciante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1690,822,2022-10-06 19:48:09.238,2022-10-06 20:02:32.385,NaN,2022-09-15,Sandra Rivera,quito,solanda,NaN,comerciante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1691,822,2022-10-06 19:48:09.238,2022-10-06 20:02:32.385,NaN,2022-09-15,Sandra Rivera,quito,solanda,NaN,comerciante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1692,822,2022-10-06 19:48:09.238,2022-10-06 20:02:32.385,NaN,2022-09-15,Sandra Rivera,quito,solanda,NaN,comerciante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df_consumidor = pd.merge(left=vecindarios_consumidor, right=composicion_hogar, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=medidas_control_plagas, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=control_malezas, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=oferta_alimentos, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=producto_dejo_producir, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=producto_vende_canal, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=canales_venta_productores, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=aplicacion_biopreparados, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=personas_trabajan_puesto_venta, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=canales_compra_comerciantes, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=productos_compra_canal, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=APOYO_RECIBIDO, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=APOYO_BRINDADO, left_on='_index', right_on='_parent_index', how='left')
df_consumidor = pd.merge(left=df_consumidor, right=HUERTO_RELACIONES, left_on='_index', right_on='_parent_index', how='left')

C:\Users\norte\AppData\Local\Temp\ipykernel_6772\1770732883.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'_parent_index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_consumidor = pd.merge(left=df_consumidor, right=oferta_alimentos, left_on='_index', right_on='_parent_index', how='left')
C:\Users\norte\AppData\Local\Temp\ipykernel_6772\1770732883.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'_parent_index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_consumidor = pd.merge(left=df_consumidor, right=producto_vende_canal, left_on='_index', right_on='_parent_index', how='left')
C:\Users\norte\AppData\Local\Temp\ipykernel_6772\1770732883.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'_parent_index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_consumidor = pd.merge(left=df_consumidor, right=aplicaci

In [55]:
df_consumidor

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,huerto_tipo_relacion/interc_produccion,huerto_tipo_relacion/interc_insumos,huerto_tipo_relacion/acopio_comercio,huerto_tipo_relacion/ensenanza,huerto_tipo_relacion/otra_relacion,huerto_otra_relacion,_2019_huerto_relacion,_2022_huerto_relacion,huertos_observaciones,_parent_index_y
0,25,2022-08-11 19:07:58.193999,2022-08-11 19:22:49.702001,NaN,2022-08-11,Rita Lema,quito,solanda,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25,2022-08-11 19:07:58.193999,2022-08-11 19:22:49.702001,NaN,2022-08-11,Rita Lema,quito,solanda,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25,2022-08-11 19:07:58.193999,2022-08-11 19:22:49.702001,NaN,2022-08-11,Rita Lema,quito,solanda,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25,2022-08-11 19:07:58.193999,2022-08-11 19:22:49.702001,NaN,2022-08-11,Rita Lema,quito,solanda,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26,2022-08-11 19:22:51.926000,2022-08-11 19:36:52.683000,NaN,2022-08-11,Rita Lema,quito,solanda,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3864,818,2022-10-06 17:55:56.364000,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3865,818,2022-10-06 17:55:56.364000,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3866,818,2022-10-06 17:55:56.364000,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3867,818,2022-10-06 17:55:56.364000,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df = pd.concat([df_productor, df_comerciante, df_consumidor], axis=0)
df

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,huerto_tipo_relacion/interc_produccion,huerto_tipo_relacion/interc_insumos,huerto_tipo_relacion/acopio_comercio,huerto_tipo_relacion/ensenanza,huerto_tipo_relacion/otra_relacion,huerto_otra_relacion,_2019_huerto_relacion,_2022_huerto_relacion,huertos_observaciones,_parent_index_y
0,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3864,818,2022-10-06 17:55:56.364,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3865,818,2022-10-06 17:55:56.364,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3866,818,2022-10-06 17:55:56.364,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3867,818,2022-10-06 17:55:56.364,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df.to_csv('primera_version_raw.csv', index=False)